# Silver: Cleaning, Normalising and DataType Conversions

In [5]:
# Import necessary functions
from pyspark.sql.types import *
from pyspark.sql.functions import when, lit, col, current_timestamp, input_file_name, lower, trim, dayofweek, month, upper
from delta.tables import *

# Configure Fabric timestamp handling (from research)
spark.conf.set("spark.sql.session.timeZone", "UTC")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")


StatementMeta(, c1858a3e-59a6-41dc-b60f-954152b3b041, 7, Finished, Available, Finished)

## Transform customers.csv into Delta table

In [2]:
# Define Schema - use DateType for signup_date
customersSchema = StructType([
    StructField("customer_id", StringType()),
    StructField("customer_name", StringType()),
    StructField("email", StringType()),
    StructField("location", StringType()),
    StructField("signup_date", DateType())
])

# Import customers file from bronze shortcut
df_customers = spark.read.format("csv").option("header","true").schema(customersSchema).load("Files/bronze_customers/customers.csv")

# Basic transformations - remove to_date since DateType() in schema handles parsing
df_customers = df_customers.withColumn("customer_name", when(col("customer_name").isNull() | (col("customer_name")==""), lit("Unknown")).otherwise(col("customer_name"))) \
    .withColumn("email", lower(trim(col("email")))) \
    .withColumn("location", when(col("location").isNull() | (col("location")==""), lit("Unknown")).otherwise(col("location")))

# Add SCD2 columns using high date approach for valid_to (standard practice)
current_ts = current_timestamp()
high_date = lit("9999-12-31 23:59:59").cast("timestamp")

df_customers = df_customers.withColumn("source_file_name", input_file_name()) \
    .withColumn("valid_from", current_ts) \
    .withColumn("valid_to", high_date) \
    .withColumn("is_active", lit(True)) \
    .withColumn("created_at", current_ts) \
    .withColumn("updated_at", current_ts)

# Display the first 10 rows to preview data
display(df_customers.head(10))

# Drop existing table if schema issues persist (uncomment if needed)
# spark.sql("DROP TABLE IF EXISTS customers_silver")

# Define the schema for the customers_silver table in Silver lakehouse
DeltaTable.createIfNotExists(spark) \
    .tableName("customers_silver") \
    .addColumn("customer_id", StringType()) \
    .addColumn("customer_name", StringType()) \
    .addColumn("email", StringType()) \
    .addColumn("location", StringType()) \
    .addColumn("signup_date", DateType()) \
    .addColumn("source_file_name", StringType()) \
    .addColumn("valid_from", TimestampType()) \
    .addColumn("valid_to", TimestampType()) \
    .addColumn("is_active", BooleanType()) \
    .addColumn("created_at", TimestampType()) \
    .addColumn("updated_at", TimestampType()) \
    .execute()

# Perform upsert operation on Delta table in Silver lakehouse
deltaTable = DeltaTable.forPath(spark, 'Tables/customers_silver')
dfUpdates = df_customers

deltaTable.alias('target') \
    .merge(
        dfUpdates.alias('source'),
        'target.customer_id = source.customer_id AND target.is_active = true'
    ) \
    .whenMatchedUpdate(
        condition="target.customer_name != source.customer_name OR target.email != source.email OR target.location != source.location",
        set = {
            "is_active": "false",
            "valid_to": "current_timestamp()",
            "updated_at": "current_timestamp()"
        }
    ) \
    .whenNotMatchedInsert(values = {
        "customer_id": "source.customer_id",
        "customer_name": "source.customer_name",
        "email": "source.email",
        "location": "source.location", 
        "signup_date": "source.signup_date",
        "source_file_name": "source.source_file_name",
        "valid_from": "source.valid_from",
        "valid_to": "source.valid_to",
        "is_active": "source.is_active",
        "created_at": "source.created_at",
        "updated_at": "source.updated_at"
    }) \
    .execute()

StatementMeta(, 137b2d74-bb18-4008-9dec-a6a12d0f5951, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 59310e28-d9ab-4b36-93fc-9e5e72d991e4)

## Transform products.csv into Delta Table

In [6]:
# Define Schema - drop ProductKey, keep ProductID as business key, use DecimalType for unit_price
productsSchema = StructType([
    StructField("product_key", IntegerType()),      # Read but will drop
    StructField("product_id", StringType()),
    StructField("product_name", StringType()),
    StructField("category", StringType()),
    StructField("stock", IntegerType()),
    StructField("unit_price", DecimalType(10, 2)),
    StructField("sales_price", DecimalType(10, 2))
])

# Import products file from bronze shortcut
df_products = spark.read.format("csv").option("header","true").schema(productsSchema).load("Files/bronze_products/products.csv")

# Basic transformations - drop ProductKey and clean data
df_products = df_products.drop("product_key") \
    .withColumn("product_name", when(col("product_name").isNull() | (col("product_name")==""), lit("Unknown")).otherwise(trim(col("product_name")))) \
    .withColumn("category", when(col("category").isNull() | (col("category")==""), lit("Uncategorized")).otherwise(trim(col("category")))) \
    .withColumn("stock", when(col("stock").isNull(), lit(0)).otherwise(col("stock"))) \
    .withColumn("unit_price", when(col("unit_price").isNull(), lit(0.00).cast(DecimalType(10, 2))).otherwise(col("unit_price"))) \
    .withColumn("sales_price", when(col("sales_price").isNull(), lit(0.00).cast(DecimalType(10, 2))).otherwise(col("sales_price"))) \
    .withColumn("is_low_stock", when(col("stock") < 20, lit(True)).otherwise(lit(False)))

# Add SCD2 columns using high date approach
current_ts = current_timestamp()
high_date = lit("9999-12-31 23:59:59").cast("timestamp")

df_products = df_products.withColumn("source_file_name", input_file_name()) \
    .withColumn("valid_from", current_ts) \
    .withColumn("valid_to", high_date) \
    .withColumn("is_active", lit(True)) \
    .withColumn("created_at", current_ts) \
    .withColumn("updated_at", current_ts)

# Display the first 10 rows to preview data
display(df_products.head(10))

# Define the schema for the products_silver table with both price columns
DeltaTable.createIfNotExists(spark) \
    .tableName("products_silver") \
    .addColumn("product_id", StringType()) \
    .addColumn("product_name", StringType()) \
    .addColumn("category", StringType()) \
    .addColumn("stock", IntegerType()) \
    .addColumn("unit_price", DecimalType(10, 2)) \
    .addColumn("sales_price", DecimalType(10, 2)) \
    .addColumn("is_low_stock", BooleanType()) \
    .addColumn("source_file_name", StringType()) \
    .addColumn("valid_from", TimestampType()) \
    .addColumn("valid_to", TimestampType()) \
    .addColumn("is_active", BooleanType()) \
    .addColumn("created_at", TimestampType()) \
    .addColumn("updated_at", TimestampType()) \
    .execute()

# Perform SCD2 upsert operation on Delta table with updated change detection
deltaTable = DeltaTable.forPath(spark, 'Tables/products_silver')
dfUpdates = df_products

deltaTable.alias('target') \
    .merge(
        dfUpdates.alias('source'),
        'target.product_id = source.product_id AND target.is_active = true'
    ) \
    .whenMatchedUpdate(
        condition="target.product_name != source.product_name OR target.category != source.category OR target.stock != source.stock OR target.unit_price != source.unit_price OR target.sales_price != source.sales_price",
        set = {
            "is_active": "false",
            "valid_to": "current_timestamp()",
            "updated_at": "current_timestamp()"
        }
    ) \
    .whenNotMatchedInsert(values = {
        "product_id": "source.product_id",
        "product_name": "source.product_name",
        "category": "source.category",
        "stock": "source.stock",
        "unit_price": "source.unit_price",
        "sales_price": "source.sales_price",
        "is_low_stock": "source.is_low_stock",
        "source_file_name": "source.source_file_name",
        "valid_from": "source.valid_from",
        "valid_to": "source.valid_to",
        "is_active": "source.is_active",
        "created_at": "source.created_at",
        "updated_at": "source.updated_at"
    }) \
    .execute()

StatementMeta(, c1858a3e-59a6-41dc-b60f-954152b3b041, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eebfad8b-eb31-48d3-a1c1-7f868d7e367b)

## Transform orders_header.csv to Delta Table

In [4]:
# Define Schema - use DecimalType for OrderTotal
ordersHeaderSchema = StructType([
    StructField("order_id", StringType()),
    StructField("customer_id", StringType()),
    StructField("order_date", DateType()),
    StructField("payment_method", StringType()),
    StructField("order_total", DecimalType(12, 2))  # Support larger order totals
])

# Import orders header file from bronze shortcut
df_orders_header = spark.read.format("csv").option("header","true").schema(ordersHeaderSchema).load("Files/bronze_pos_orders_header/orders_header.csv")

# Basic transformations - focus on data quality and temporal features
df_orders_header = df_orders_header \
    .withColumn("payment_method", when(col("payment_method").isNull() | (col("payment_method")==""), lit("Unknown")).otherwise(upper(trim(col("payment_method"))))) \
    .withColumn("order_total", when(col("order_total").isNull() | (col("order_total") <= 0), lit(0.00).cast(DecimalType(12, 2))).otherwise(col("order_total"))) \
    .withColumn("order_day_of_week", dayofweek(col("order_date"))) \
    .withColumn("order_month", month(col("order_date")))

# Add audit columns (no SCD2 high date approach for SCD1)
current_ts = current_timestamp()

df_orders_header = df_orders_header.withColumn("source_file_name", input_file_name()) \
    .withColumn("created_at", current_ts) \
    .withColumn("updated_at", current_ts)

# Display the first 10 rows to preview data
display(df_orders_header.head(10))

# Define the schema for the orders_header_silver table (SCD1 - no temporal columns)
DeltaTable.createIfNotExists(spark) \
    .tableName("orders_header_silver") \
    .addColumn("order_id", StringType()) \
    .addColumn("customer_id", StringType()) \
    .addColumn("order_date", DateType()) \
    .addColumn("payment_method", StringType()) \
    .addColumn("order_total", DecimalType(12, 2)) \
    .addColumn("order_day_of_week", IntegerType()) \
    .addColumn("order_month", IntegerType()) \
    .addColumn("source_file_name", StringType()) \
    .addColumn("created_at", TimestampType()) \
    .addColumn("updated_at", TimestampType()) \
    .execute()

# Perform SCD1 upsert operation on Delta table with change detection
deltaTable = DeltaTable.forPath(spark, 'Tables/orders_header_silver')
dfUpdates = df_orders_header

deltaTable.alias('target') \
    .merge(
        dfUpdates.alias('source'),
        'target.order_id = source.order_id'
    ) \
    .whenMatchedUpdate(
        condition="target.payment_method != source.payment_method OR target.order_total != source.order_total",
        set = {
            "customer_id": "source.customer_id",
            "order_date": "source.order_date",
            "payment_method": "source.payment_method",
            "order_total": "source.order_total",
            "order_day_of_week": "source.order_day_of_week",
            "order_month": "source.order_month",
            "updated_at": "current_timestamp()"
        }
    ) \
    .whenNotMatchedInsert(values = {
        "order_id": "source.order_id",
        "customer_id": "source.customer_id",
        "order_date": "source.order_date",
        "payment_method": "source.payment_method",
        "order_total": "source.order_total",
        "order_day_of_week": "source.order_day_of_week",
        "order_month": "source.order_month",
        "source_file_name": "source.source_file_name",
        "created_at": "source.created_at",
        "updated_at": "source.updated_at"
    }) \
    .execute()

StatementMeta(, 137b2d74-bb18-4008-9dec-a6a12d0f5951, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 26062382-6213-40d2-b370-c0734df14ac8)

## Transform order_lines.csv into Delta Table

In [5]:
# Define Schema - use DecimalType for all monetary fields
orderLinesSchema = StructType([
    StructField("order_line_id", StringType()),
    StructField("order_id", StringType()),
    StructField("line_number", IntegerType()),
    StructField("product_id", StringType()),
    StructField("quantity", IntegerType()),
    StructField("unit_price", DecimalType(10, 2)),
    StructField("discount_amount", DecimalType(10, 2)),
    StructField("tax_amount", DecimalType(10, 2)),
    StructField("line_total", DecimalType(12, 2))
])

# Import order lines file from bronze shortcut
df_order_lines = spark.read.format("csv").option("header","true").schema(orderLinesSchema).load("Files/bronze_pos_order_lines/order_lines.csv")

# Basic transformations - data quality and validation
df_order_lines = df_order_lines \
    .withColumn("quantity", when(col("quantity").isNull() | (col("quantity") <= 0), lit(1)).otherwise(col("quantity"))) \
    .withColumn("unit_price", when(col("unit_price").isNull(), lit(0.00).cast(DecimalType(10, 2))).otherwise(col("unit_price"))) \
    .withColumn("discount_amount", when(col("discount_amount").isNull(), lit(0.00).cast(DecimalType(10, 2))).otherwise(col("discount_amount"))) \
    .withColumn("tax_amount", when(col("tax_amount").isNull(), lit(0.00).cast(DecimalType(10, 2))).otherwise(col("tax_amount"))) \
    .withColumn("line_total", when(col("line_total").isNull(), lit(0.00).cast(DecimalType(12, 2))).otherwise(col("line_total")))

# Add audit columns
current_ts = current_timestamp()

df_order_lines = df_order_lines.withColumn("source_file_name", input_file_name()) \
    .withColumn("created_at", current_ts) \
    .withColumn("updated_at", current_ts)

# Display the first 10 rows to preview data
display(df_order_lines.head(10))

# Define the schema for the order_lines_silver table (SCD1)
DeltaTable.createIfNotExists(spark) \
    .tableName("order_lines_silver") \
    .addColumn("order_line_id", StringType()) \
    .addColumn("order_id", StringType()) \
    .addColumn("line_number", IntegerType()) \
    .addColumn("product_id", StringType()) \
    .addColumn("quantity", IntegerType()) \
    .addColumn("unit_price", DecimalType(10, 2)) \
    .addColumn("discount_amount", DecimalType(10, 2)) \
    .addColumn("tax_amount", DecimalType(10, 2)) \
    .addColumn("line_total", DecimalType(12, 2)) \
    .addColumn("source_file_name", StringType()) \
    .addColumn("created_at", TimestampType()) \
    .addColumn("updated_at", TimestampType()) \
    .execute()

# Perform SCD1 upsert operation on Delta table (update in place)
deltaTable = DeltaTable.forPath(spark, 'Tables/order_lines_silver')
dfUpdates = df_order_lines

deltaTable.alias('target') \
    .merge(
        dfUpdates.alias('source'),
        'target.order_line_id = source.order_line_id'
    ) \
    .whenMatchedUpdate(set = {
        "order_id": "source.order_id",
        "line_number": "source.line_number",
        "product_id": "source.product_id",
        "quantity": "source.quantity",
        "unit_price": "source.unit_price",
        "discount_amount": "source.discount_amount",
        "tax_amount": "source.tax_amount",
        "line_total": "source.line_total",
        "updated_at": "current_timestamp()"
    }) \
    .whenNotMatchedInsert(values = {
        "order_line_id": "source.order_line_id",
        "order_id": "source.order_id",
        "line_number": "source.line_number",
        "product_id": "source.product_id",
        "quantity": "source.quantity",
        "unit_price": "source.unit_price",
        "discount_amount": "source.discount_amount",
        "tax_amount": "source.tax_amount",
        "line_total": "source.line_total",
        "source_file_name": "source.source_file_name",
        "created_at": "source.created_at",
        "updated_at": "source.updated_at"
    }) \
    .execute()

StatementMeta(, 137b2d74-bb18-4008-9dec-a6a12d0f5951, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3293f2db-0abd-4f15-8950-d59eca812596)